<a href="https://colab.research.google.com/github/marcelo7bastos/mba_mcdia_est_desc_analise_pronaf/blob/main/mba_mcdia_est_desc_analise_pronaf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Projeto Final - Estatística Descritiva**


## **O acesso ao crédito do PRONAF resultou em melhoria nas condições econômicas dos agricultores familiares ao longo de 10 anos?**

# Preparação do Ambiente e Dados

In [1]:
# Bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Define o caminho da pasta onde os arquivos serão armazenados/recuperados
drive_folder = '/content/drive/MyDrive/ENAP_MBA_MCDIA/005_estatistica/trabalho_final/data'

# Parâmetros da análise
MODO_ANALISE = "amostra"  # Opções: "amostra", "nova_amostra", "populacao"
FRAC_AMOSTRA = 0.1  # 10% da base para amostragem

# Estrutura de decisão para carregar os dados
if MODO_ANALISE == "amostra":
    print("🔹 Carregando apenas uma amostra do dataset...")
    # Caminho para o arquivo de amostra já salvo no Google Drive
    arquivo_parquet = f"{drive_folder}/pronaf_amostra.parquet"
    df = pd.read_parquet(arquivo_parquet)

elif MODO_ANALISE == "populacao":
    print("🔹 Carregando a população completa (isso pode demorar)...")
    arquivo_parquet = f"{drive_folder}/pronaf.parquet"
    # Carrega o arquivo
    df = pd.read_parquet(arquivo_parquet)

else:
    raise ValueError("Modo de análise inválido! Escolha entre: 'amostra', 'nova_amostra' ou 'populacao'.")

# Exibir informações do dataset carregado
print(f"Número total de registros carregados: {df.shape[0]:,}".replace(",", "."))



🔹 Carregando apenas uma amostra do dataset...
Número total de registros carregados: 2.574.808


In [4]:
# Dicionário com as conversões recomendadas para cada variável
conversoes = {
    'REF_BACEN': 'str',  # Código identificador, manter como string
    'NU_ORDEM': 'str',  # Número de ordem, mas é um identificador também, manter string
    #'CNPJ_IF': 'str',  # Identificador de instituição financeira
    'DT_EMISSAO': 'datetime64[ns]',  # Data de emissão do contrato
    #'DT_VENCIMENTO': 'datetime64[ns]',  # Data de vencimento do contrato
    'CD_INST_CREDITO': 'str',  # Código da instituição financeira, manter string
    #'CD_CATEG_EMITENTE': 'str',  # Categoria do emitente
    'CD_FONTE_RECURSO': 'str',  # Código da fonte de recurso, manter string
    #'CNPJ_AGENTE_INVEST': 'str',  # Identificador do agente investidor
    'CD_ESTADO': 'str',  # Código de estado (UF), manter string
    #'CD_REF_BACEN_INVESTIMENTO': 'str',  # Código de referência Bacen para investimento
    #'CD_TIPO_SEGURO': 'str',  # Código do tipo de seguro
    #'CD_EMPREENDIMENTO': 'str',  # Código do empreendimento
    'CD_PROGRAMA': 'str',  # Código do programa de crédito
    #'CD_TIPO_ENCARG_FINANC': 'str',  # Tipo de encargo financeiro
    #'CD_TIPO_IRRIGACAO': 'str',  # Tipo de irrigação utilizada
    #'CD_TIPO_AGRICULTURA': 'str',  # Tipo de agricultura praticada
    'CD_FASE_CICLO_PRODUCAO': 'str',  # Fase do ciclo de produção, manter string
    #'CD_TIPO_CULTIVO': 'str',  # Tipo de cultivo agrícola
    #'CD_TIPO_INTGR_CONSOR': 'str',  # Tipo de integração consorciada
    #'CD_TIPO_GRAO_SEMENTE': 'str',  # Tipo de grão/semente plantado
    #'VL_ALIQ_PROAGRO': 'float64',  # Percentual de alíquota Proagro
    'VL_JUROS': 'float64',  # Taxa de juros, valor decimal
    #'VL_PRESTACAO_INVESTIMENTO': 'float64',  # Valor da prestação de investimento
    #'VL_PREV_PROD': 'float64',  # Valor previsto da produção
    #'VL_QUANTIDADE': 'float64',  # Quantidade financiada
    #'VL_RECEITA_BRUTA_ESPERADA': 'float64',  # Receita bruta esperada
    'VL_PARC_CREDITO': 'float64',  # Valor da parcela de crédito
    #'VL_REC_PROPRIO': 'float64',  # Receita própria do produtor
    #'VL_PERC_RISCO_STN': 'float64',  # Percentual de risco do STN
    #'VL_PERC_RISCO_FUNDO_CONST': 'float64',  # Percentual de risco do fundo
    #'VL_REC_PROPRIO_SRV': 'float64',  # Receita própria de serviço
    #'VL_AREA_FINANC': 'float64',  # Área financiada em hectares
    'CD_SUBPROGRAMA': 'str',  # Código do subprograma de financiamento
    #'VL_PRODUTIV_OBTIDA': 'float64',  # Produtividade obtida na lavoura
    #'DT_FIM_COLHEITA': 'datetime64[ns]',  # Data fim da colheita
    #'DT_FIM_PLANTIO': 'datetime64[ns]',  # Data fim do plantio
    #'DT_INIC_COLHEITA': 'datetime64[ns]',  # Data início da colheita
    #'DT_INIC_PLANTIO': 'datetime64[ns]',  # Data início do plantio
    #'VL_JUROS_ENC_FINAN_POSFIX': 'float64',  # Juros pós-fixados
    #'VL_PERC_CUSTO_EFET_TOTAL': 'float64',  # Percentual de custo efetivo total
    #'CD_CONTRATO_STN': 'str',  # Código do contrato STN
    #'CD_CNPJ_CADASTRANTE': 'str',  # CNPJ do cadastrante
    'VL_AREA_INFORMADA': 'float64',  # Área informada em hectares
    #'CD_CICLO_CULTIVAR': 'str',  # Ciclo da cultivar (curto, médio, longo)
    #'CD_TIPO_SOLO': 'str',  # Tipo de solo utilizado
    #'PC_BONUS_CAR': 'float64',  # Percentual de bônus CAR
    #'MES': 'Int64',  # Número do mês
    #'SEMESTRE': 'Int64',  # Número do semestre
    'ANO': 'Int64',  # Ano do financiamento
    #'SAFRA': 'str',  # Safra agrícola
    #'PROGRAMA_DESCRICAO': 'str',  # Descrição do programa
    #'CD_IBGE_MUNICIPIO': 'str',  # Código IBGE do município
    'CD_CPF_CNPJ': 'str',  # CPF/CNPJ do tomador de crédito
    'SEXO_BIOLOGICO': 'category'  # Masculino ou Feminino
    #'CD_DAP': 'str', #'Código do DAP'
}

# Filtrar apenas as colunas de interesse
df = df[list(conversoes.keys())]

# Aplicar conversão ao DataFrame
df = df.astype(conversoes)

# Exibir tipos para conferir a conversão
print(df.dtypes)

REF_BACEN                         object
NU_ORDEM                          object
DT_EMISSAO                datetime64[ns]
CD_INST_CREDITO                   object
CD_FONTE_RECURSO                  object
CD_ESTADO                         object
CD_PROGRAMA                       object
CD_FASE_CICLO_PRODUCAO            object
VL_JUROS                         float64
VL_PARC_CREDITO                  float64
CD_SUBPROGRAMA                    object
VL_AREA_INFORMADA                float64
ANO                                Int64
CD_CPF_CNPJ                       object
SEXO_BIOLOGICO                  category
dtype: object
